In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import xgboost as xgb
from xgboost import XGBRegressor
import os
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import joblib
import warnings
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC,ElasticNetCV,LassoCV
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score, train_test_split
import xgboost as xgb
import lightgbm as lgb
import featuretools as ft
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.metrics import mean_squared_error
from xgboost import plot_importance
from datetime import datetime
import datetime
from scipy.special import boxcox1p
from sklearn.metrics import r2_score
import lightgbm as lgb
color = sns.color_palette()
sns.set_style('darkgrid')
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)


from scipy import stats
from scipy.stats import norm, skew #for some statistics

pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points

In [48]:
def plot_predictions(y, yhat, title="Predictions vs Actual", output_dir=None):
    """
    Plot the predictions against the actual values

    Parameters
    -----
    y : actual (real) values

    yhat : predicted values

    title : plot title
    """

    fig = plt.figure(figsize=(15, 6))
    plt.xlabel('Time')
    plt.ylabel('PM10')
    plt.plot(y, label="actual", figure=fig)
    plt.plot(yhat, label="predicted", figure=fig)
    plt.title(title)
    fig.legend()

    if output_dir != None:
        plt.savefig(os.path.join(output_dir, "{}.png".format(title)))

    plt.close(fig)

def train_and_evaluate(name, model, train,final_eval, output_dir, predictors):
    """
    Train and evaluate a model through the scikit-learn pipeline

    A pipeline is build using the supplied model as the final step in the pipeline (see make_pipeline)
    Then, the input data is transformed accordingly to be able to fit into the pipeline.

    The model is saved after the last step under `output_dir`/models with the filename `name`.joblib,
    if you wish to load them later and reevaluate/retrain them.

    Parameters
    -----
    name : the name of the model

    model : a scikit-learn model object

    train : train portion of the dataset

    final_eval : last 2*X:X hours from the dataset

    output_dir : base directory for saving output files

    Returns
    -----
    ret : mean absolute error of the trained model on last X hours
    """

    print("---" * 5)
    print("Running pipeline for {}".format(name))

    plot_dir = os.path.join(output_dir, "plots")

    pipeline = model

    X_train, y_train = train.drop(
        ["PM10"], axis=1)[predictors].values, train["PM10"].values
    X_final, y_final = final_eval.drop(
        ["PM10"], axis=1)[predictors].values, final_eval["PM10"].values
    print("Fitting pipeline on all \"all available data\"")
    
    pipeline.fit(X_train, y_train)
    yhat = pipeline.predict(X_final)
    yhat_return = np.expm1(yhat)
    y_final_return = np.expm1(y_final)
    mae = mean_absolute_error(y_final_return, yhat_return)
    print("MAE: {}".format(mae))
    plot_predictions(y_final_return, yhat_return, title="{} - Predicted vs. Actual".format(name), output_dir=plot_dir)

    # save the model
    joblib.dump(model, os.path.join(
        output_dir, "models", "{}.joblib".format(name)))

    return yhat_return, mae

In [49]:
#Simple stacking
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)  
    
averaged_models = AveragingModels(models = (ENet, GBoost, lasso,model_xgb,model_lgb))

score = mae(averaged_models,predictors)
print(" Averaged base models score: {:.4f} \n",score)


# STACKING WITH A META MODEL

class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)
    

In [52]:
def read_csv_series(path, ts_column="time"):
    """
    Read a time series from a CSV file.

    The CSV file must contain a column with either a UNIX timestamp or a datetime
    string with any format supported by Pandas. 

    Parameters
    -----
    path : path to CSV file

    ts_column : name of the column containing time data, "time" by default

    Returns
    -----
    ret : Pandas Series object with datetime as index
    """

    # read CSV
    df = pd.read_csv(path, parse_dates=[ts_column])
    # convert timestamps to datetime objects using panda's to_datetime
    df[ts_column] = pd.to_datetime(df[ts_column], unit="s")
    # set datetime as index (make time series)
    df.index = df[ts_column]
    # delete original time column
    del df[ts_column]

    # remove rows with duplicated time if there are any, keep first duplicate row
    df = df.loc[~df.index.duplicated(keep="first")]

    df.index.name = ts_column

    return df


def make_directory_tree(tree, output_dir):
    """
    Create the output directory tree structure specified by `tree` in `output_dir`

    Parameters
    -----
    tree : list of paths to create under `output_dir`

    output_dir : path to root of output directory tree
    """

    for d in tree:
        try:
            path = os.path.join(output_dir, d)
            os.makedirs(path, exist_ok=True)
        except OSError as e:
            if e.errno == errno.EEXIST and os.path.isdir(path):
                print("Path already exists: {}".format(d))
                print("Files may be overwritten")
                continue
            else:
                raise


def get_datasets(data_dir):
    """
    Get all .csv filenames from the specified directory

    Parameters
    -----
    data_dir : path to directory containing .csv files

    Returns
    -----
    ret : list containing dataset filenames
    """

    return [f for f in os.listdir(data_dir) if os.path.isfile(
        os.path.join(data_dir, f)) and f.endswith(".csv")]


def run(input_dir, output_dir, models, model_names, team_name="OrganizersTeam", predict_window=8760):
    """
    Train and evaluate models for each dataset under `input_dir`

    This script trains and evaluates 6 models and ensembles them using meta model stacking
    1. Lasso
    2. ENet
    3. KernelRidge
    4. GradientBoostingRegressor
    5. XGB
    6. LGB
    7. Meta stacking of previously mentioned models, LGB acting as the meta

    Parameters
    -----
    input_dir : directory containing datasets

    output_dir : directory for saving useful output files (models, etc)

    predict_window : number of hours needed to predict, default=12

    Returns
    -----
    ret : a Pandas DataFrame containing the scores for each trained model
    """

    models_dir = os.path.join(output_dir, "models")
    plots_dir = os.path.join(output_dir, "plots")
    sub_dir = os.path.join(output_dir, "submissions")
    submission_file_name_fmt = "{}_{}.csv"

    make_directory_tree(["models","plots", "submissions"], output_dir)

    datasets = get_datasets(input_dir)

    # create a scores table to keep MAE for each location:model pair
    scores = pd.DataFrame(columns=["Location", "Model", "MAE"])

    for dataset in datasets:
        # load the dataset
        df = read_csv_series(os.path.join(input_dir, dataset))
        loc = dataset.split(".")[0]

        # shift PM10 for `predict_window` hours ahead
        df["PM10"] = df["PM10"].shift(-predict_window)

        # split dataset into train, test and evaluation by dates
        train_len = int((len(df) - (2 * predict_window)))
        train= df[:train_len]
        final_eval = df[-(2 * predict_window):-predict_window].copy()
        
        # initialize model, PUT OPTIMIZED MODELS HERE
        print("Will train a total of {} models".format(len(models) * 1))

        mae_min = 1e10
        yhat_sub = []
        predictors = [x for x in df.columns if x not in ["PM10","time"]]
        
        for model_name,model in zip(model_names,models):
            # get predictions and MAE
            yhat, mae = train_and_evaluate("{} - {}".format(loc, model_name), model, train,
                                           final_eval, output_dir, predictors)

            # save the score (MAE) for the model
            scores = scores.append(
                {"Location": loc, "Model": model_name, "MAE": mae}, ignore_index=True)

            # save the better predictions to `yhat_sub`
            if mae < mae_min:
                mae_min = mae
                yhat_sub = yhat

        sub_df = pd.DataFrame(yhat_sub, columns=["PM10"])
        sub_df.to_csv(os.path.join(
            sub_dir, submission_file_name_fmt.format(team_name, loc)))

        scores.to_csv(os.path.join(output_dir, "scores.csv"))

    print("Done")
    print("Saved models can be found at {}".format(models_dir))
    print("Plots can be found at {}".format(plots_dir))
    print("Submissions can be found at {}".format(sub_dir))

    return scores

In [1]:
output_dir = 'pm10_scores'

In [2]:
# Testing lasso, enet, gboost, xgboost, lgboost and a stacking model with lgboost as meta

lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
enet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))
gboost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                       max_depth=4, max_features='sqrt',
                       min_samples_leaf=15, min_samples_split=10, 
                       loss='huber', random_state =5)
xgboost = xgb.XGBRegressor(learning_rate =0.1, # OTPIMIZED, not to be changed
                           n_estimators=1200,
                           max_depth=2,
                           min_child_weight=0.5,
                           gamma=0,
                           subsample=0.8,
                           n_jobs=-1,
                           colsample_bytree=0.8,
                           reg_alpha=0.01,
                           random_state=42)
models=[lasso,enet,gboost,xgboost]

lgboost = lgb.LGBMRegressor(objective='regression',num_leaves=5, # THE META MODEL
                      learning_rate=0.05, n_estimators=720,
                      max_bin = 55, bagging_fraction = 0.8,
                      bagging_freq = 5, feature_fraction = 0.2319,
                      feature_fraction_seed=9, bagging_seed=9,
                      min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

stacked_averaged_models = StackingAveragedModels(base_models = models,
                                                 meta_model = lgboost)

final_models = [lasso, enet, xgboost, lgboost, stacked_averaged_models]
model_names = ["lasso","enet","xgboost","lgboost","stacked_model_with_lgboost_meta"]
# input_folder = "blabla"
# run(input_folder, output_dir, final_models, model_names)

NameError: name 'make_pipeline' is not defined

In [ ]:
# Testing Ridge, SVM, decision regression tree, random forest, Multi-layer Perceptron regressor

krr = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
svr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
tree_regressor = DecisionTreeRegressor(random_state=0)
forest_regressor = RandomForestRegressor(random_state=0)
mlp_regr = MLPRegressor(random_state=1, max_iter=500)

final_models = [krr, svr, tree_regressor, forest_regressor, mlp_regr]
model_names = ["kernel_ridge", "svr", "tree_regressor", "forest_regressor", "mlp_regressor"]
# input_folder = "blabla"
# run(input_folder, output_dir, final_models, model_names)

In [ ]:
# Testing LSTM

In [ ]:
# Testing ARIMA